In [1]:
import numpy as np
import pandas as pd
import gamspy as gp
import sys
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [3]:
n = 8
m = 5
D1 = np.random.binomial(10, 0.5, n)
D2 = np.random.binomial(10, 0.5, n)
print(D1)
print(D2)

[2 5 5 3 5 3 5 4]
[5 4 7 6 6 4 4 7]


In [62]:
b = np.array([1, 2, 3, 4, 5])
s = np.array([1, 1, 1, 1, 1])
c = np.random.randint(low=-50, high=-40, size=8)
c

array([-44, -47, -44, -45, -48, -47, -42, -46])

In [63]:
matrix_A= np.zeros(m*n).reshape(m, n)
for i in range(m):
    for j in range(n):
        matrix_A[i][j] = np.random.randint(5)
matrix_A

array([[0., 0., 1., 2., 2., 0., 2., 4.],
       [3., 3., 2., 2., 4., 3., 0., 4.],
       [2., 1., 4., 3., 4., 3., 3., 3.],
       [0., 4., 1., 3., 4., 3., 2., 0.],
       [3., 0., 2., 3., 2., 2., 4., 4.]])

In [64]:
set_n = np.arange(n)
set_m = np.arange(m)

In [65]:
model = Container()
i = Set(container=model, name='set_n', records=set_n)
j = Set(container=model, name='set_m', records=set_m)

In [66]:
B = Parameter(
        container=model,
        name='b',
        domain=j,
        description='coefficient of objective function',
        records=b,
)

In [67]:
S = Parameter(
    container=model,
    name='S',
    domain=j,
    description='savage',
    records=s,
)
S.records

,set_m,value
0,0,1.0
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0


In [68]:
C = Parameter(
    container=model,
    name='C',
    domain=i,
    description='C',
    records=c,
)
C.records

,set_n,value
0,0,-44.0
1,1,-47.0
2,2,-44.0
3,3,-45.0
4,4,-48.0
5,5,-47.0
6,6,-42.0
7,7,-46.0


In [69]:
d1 = Parameter(
        container=model,
        name='d1',
        domain=i,
        description='demand_1',
        records=D1,
)
d2 = Parameter(
        container=model,
        name='d2',
        domain=i,
        description='demand_2',
        records=D2,
)

In [70]:
A = Parameter(
        container=model,
        name="A",
        domain=[j, i],
        description="matrix A",
        records=matrix_A,
)

In [71]:
x = Variable(
        container=model,
        name="x",
        domain=j,
        type="Positive",
        description="x_decision variable",
)
y1 = Variable(
        container=model,
        name='y1',
        domain=j,
        type='Positive',
        description='y1_decision variable',
)
y2 = Variable(
        container=model,
        name='y2',
        domain=j,
        type='Positive',
        description='y2_decision variable',
)
z1 = Variable(
        container=model,
        name='z1',
        domain=i,
        type='Positive',
        description='z1_decision variable',
)
z2 = Variable(
        container=model,
        name='z2',
        domain=i,
        type='Positive',
        description='z2_decision variable',
)

In [72]:
z1_ctr = Equation(
        container=model, name="z1_constraints", domain=i, description="z1ctr"
)
z1_ctr[i] = z1[i] <= d1[i]
z2_ctr = Equation(
        container=model, name="z2_constraints", domain=i, description="z2ctr"
)
z2_ctr[i] = z2[i] <= d2[i]
y1_ctr = Equation(
        container=model, name="y1_constraints", domain=j, description="c1ctr"
)
y1_ctr[j] = y1[j] == x[j] - Sum(i, A[j, i]*z1[i])
y2_ctr = Equation(
        container=model, name="y2_constraints", domain=j, description="c2ctr"
)
y2_ctr[j] = y2[j] == x[j] - Sum(i, A[j, i]*z2[i])

In [73]:
obj = Sum(j, B[j]*x[j]) + 0.5*(Sum(i, C[i] * z1[i]) - Sum(j, S[j] * y1[j])) + 0.5*(Sum(i, C[i] * z2[i]) - Sum(j, S[j] * y2[j]))

In [74]:
transport = Model(
        model,
        name="linear_programming",
        equations=[z1_constraints, z2_constraints, y1_constraints, y2_constraints],
        problem="LP",
        sense=Sense.MIN,
        objective=obj,
)   
transport.solve(output=sys.stdout)
transport.objective_value

--- Job _job_e6250a27-4861-4b11-9ccb-1becf1dc9acf.gms Start 11/24/23 21:22:19 45.3.0 c4c31d8a WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Input C:\Users\Admin\AppData\Local\Temp\tmpurs6xlpk\_job_e6250a27-4861-4b11-9ccb-1becf1dc9acf.gms
    Output C:\Users\Admin\AppData\Local\Temp\tmpurs6xlpk\_job_e6250a27-4861-4b11-9ccb-1becf1dc9acf.lst
    Save C:\Users\Admin\AppData\Local\Temp\tmpurs6xlpk\_save_b6dad25e-db59-4d4c-bc0e-7c4c8a4df067.g00
    ScrDir C:\Users\Admin\AppData\Local\Temp\tmpurs6xlpk\225a\
    SysDir C:\Users\Admin\AppData\Roaming\Python\Python311\site-packages\gamspy_base\
    CurDir C:\Users\Admin\AppData\Local\Temp\tmpurs6xlpk\
 

-305.625